In [16]:
from shapely.geometry import Point, Polygon, MultiPolygon
from plotly import graph_objects as go
from dotenv import load_dotenv
from unidecode import unidecode
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import requests
import os

In [17]:
%matplotlib inline

sns.set_style('darkgrid')

load_dotenv()

True

In [18]:
apartments = pd.read_csv('../data/interim/apartments.csv')
apartments.head(3)

,codigo,tipo_propiedad,tipo_operacion,precio_venta,precio_arriendo,area,habitaciones,banos,administracion,parqueaderos,...,piso,closets,chimenea,permite_mascotas,gimnasio,ascensor,conjunto_cerrado,coords_modified,localidad,barrio
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2.500000e+09,13500000.0,320.0,4.0,5.0,2500000.0,3.0,...,8.0,4.0,1,1,1,1,1,False,USAQUEN,CIUDADELA REAL
1,2510-M4156145,APARTAMENTO,VENTA,4.850000e+08,NaN,84.0,3.0,3.0,680000.0,1.0,...,2.0,3.0,0,0,0,1,1,False,USAQUEN,SAN PATRICIO
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1.980000e+09,9000000.0,248.0,3.0,3.0,2200000.0,3.0,...,3.0,NaN,1,1,0,1,1,False,USAQUEN,CERROS DE SANTA BARBARA


In [19]:
apartments.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'descripcion', 'datetime', 'website', 'compañia',
       'fecha_actualizacion_precio_venta', 'precio_venta_anterior',
       'fecha_actualizacion_precio_arriendo', 'precio_arriendo_anterior',
       'jacuzzi', 'piso', 'closets', 'chimenea', 'permite_mascotas',
       'gimnasio', 'ascensor', 'conjunto_cerrado', 'coords_modified',
       'localidad', 'barrio'],
      dtype='object')

In [20]:
response = requests.get('https://gis.transmilenio.gov.co/arcgis/rest/services/Troncal/consulta_estaciones_troncales/FeatureServer/1/query?where=1%3D1&outFields=*&f=json').json()
troncal_transmilenio = pd.DataFrame(response['features'])
troncal_transmilenio = pd.json_normalize(troncal_transmilenio['attributes'])
troncal_transmilenio.head(3)

,objectid,numero_estacion,nombre_estacion,coordenada_x_estacion,coordenada_y_estacion,ubicacion_estacion,troncal_estacion,numero_vagones_estacion,numero_accesos_estacion,biciestacion_estacion,...,longitud_estacion,globalid,created_user,created_date,last_edited_user,last_edited_date,codigo_nodo_estacion,componente_wifi,componente_aplificacion,log_replica
0,116,07001,Perdomo,990294.7280,9.999536e+05,AutoSur-Kr78C,NQS,2,1,0,...,-74.164971,{61AAA51C-F036-4D99-B895-E3DCA2F52D47},None,None,None,None,7001,Valor SI al campo,None,None
1,87,09110,AV. Jiménez - CL 13,999822.3829,1.000757e+06,Calle 13 - caracas,Americas,3,2,0,...,-74.079109,{F4282057-CEBC-431E-9178-602615311D01},None,None,None,None,14003,Valor SI al campo,None,None
2,71,09110,AV. Jiménez - Caracas,999675.9713,1.000738e+06,Calle 13 - Calle 11,Caracas,4,2,0,...,-74.080428,{D830EF50-FC3D-416D-BE9A-9A13550841A3},None,None,None,None,9110,Valor SI al campo,None,None


In [21]:
troncal_transmilenio.columns

Index(['objectid', 'numero_estacion', 'nombre_estacion',
       'coordenada_x_estacion', 'coordenada_y_estacion', 'ubicacion_estacion',
       'troncal_estacion', 'numero_vagones_estacion',
       'numero_accesos_estacion', 'biciestacion_estacion',
       'capacidad_biciestacion_estacion', 'tipo_estacion',
       'biciparqueadero_estacion', 'latitud_estacion', 'longitud_estacion',
       'globalid', 'created_user', 'created_date', 'last_edited_user',
       'last_edited_date', 'codigo_nodo_estacion', 'componente_wifi',
       'componente_aplificacion', 'log_replica'],
      dtype='object')

In [22]:
def normalize(text):
    try:
        return unidecode(text).upper()
    except:
        return text

# Agregar datos acerca de las troncales de transmilenio

In [23]:
import math

def haversine_m(lat1, lon1, lat2, lon2):
    r = 6371000 # radio de la tierra en metros

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Fórmula del haversine
    a = math.sin(delta_lat/2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distancia = r * c
    
    return distancia

In [24]:
def estacion_tm_cercana(row):
    try:
        distancias = []
        for i, estacion in troncal_transmilenio.iterrows():
            distancias.append(haversine_m(row['latitud'], row['longitud'], estacion['latitud_estacion'], estacion['longitud_estacion']))
        return troncal_transmilenio.loc[np.argmin(distancias), 'nombre_estacion']
    except Exception as e:
        print(e)
        return np.nan
    
def get_distancia_estacion_m(row):
    try:
        distancias = []
        for i, estacion in troncal_transmilenio.iterrows():
            distancias.append(haversine_m(row['latitud'], row['longitud'], estacion['latitud_estacion'], estacion['longitud_estacion']))
        return min(distancias)
    except:
        return np.nan
    
apartments['estacion_tm_cercana'] = apartments.apply(estacion_tm_cercana, axis=1)
apartments['distancia_estacion_tm_m'] = apartments.apply(get_distancia_estacion_m, axis=1)

In [25]:
apartments['distancia_estacion_tm_m'] = apartments['distancia_estacion_tm_m'].apply(lambda x: round(x, 2))
apartments['distancia_estacion_tm_m'].describe()

count    39231.000000
mean      1259.765847
std        828.967991
min          1.530000
25%        569.305000
50%       1172.550000
75%       1707.380000
max       7206.140000
Name: distancia_estacion_tm_m, dtype: float64

In [26]:
fig = px.scatter_mapbox(
    apartments.loc[apartments['distancia_estacion_tm_m'] <= 500],
    lat='latitud', 
    lon='longitud', 
    color='estrato', 
    zoom=10, 
    height=500,
    hover_name='barrio',
    hover_data=['area', 'habitaciones', 'banos', 'estrato', 'localidad', 'distancia_estacion_tm_m', 'estacion_tm_cercana'],
    color_continuous_scale=px.colors.sequential.Viridis,
)

fig.add_trace(
    go.Scattermapbox(
        lat=troncal_transmilenio['latitud_estacion'],
        lon=troncal_transmilenio['longitud_estacion'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='red',
            opacity=0.7
        ),
        hoverinfo='text',
        text=troncal_transmilenio['nombre_estacion'],
        showlegend=False
    )
)

fig.update_layout(
    mapbox=dict(
        accesstoken=os.getenv('MAPBOX_TOKEN'),
        style='dark',
        center=dict(
            lat=4.60971,
            lon=-74.08175
        )
    )
)

fig.update_layout(margin={'r': 0, 't': 0, 'l': 0, 'b': 0})

fig.show()

In [27]:
def is_cerca_estacion(row):
    if row['distancia_estacion_tm_m'] <= 500:
        return 1
    else:
        return 0
    
apartments['cerca_estacion_tm'] = apartments.apply(is_cerca_estacion, axis=1)

In [28]:
apartments.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'descripcion', 'datetime', 'website', 'compañia',
       'fecha_actualizacion_precio_venta', 'precio_venta_anterior',
       'fecha_actualizacion_precio_arriendo', 'precio_arriendo_anterior',
       'jacuzzi', 'piso', 'closets', 'chimenea', 'permite_mascotas',
       'gimnasio', 'ascensor', 'conjunto_cerrado', 'coords_modified',
       'localidad', 'barrio', 'estacion_tm_cercana', 'distancia_estacion_tm_m',
       'cerca_estacion_tm'],
      dtype='object')

In [29]:
apartments.to_csv('../data/processed/apartments.csv', index=False)

In [30]:
import session_info

session_info.show()